# Анализ работы API (CPU vs GPU)

In [8]:
import os
import requests
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import time

## CPU

### CPU - без переключения на device
> CPU - 10 CPU & 16 RAM

#### Английский

In [11]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [12]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

### CPU - используется torch.device('cpu')
> CPU - 10 CPU & 16 RAM

#### Английский

In [13]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [14]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

### Увеличим CPU и RAM
> CPU - 16 CPU & 64 RAM

#### Английский

In [19]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [20]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

### Уменьшим CPU и оставим RAM таким же
> CPU - 10 CPU & 64 RAM

#### Английский

In [21]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [22]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

## GPU
> CPU - 10 CPU & 16 RAM

### GPU - используется при переводе одного предложения

In [1]:
import os
import requests
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import time

#### Английский

In [4]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [5]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

### GPU - используется при инициализации
> CPU - 10 CPU & 16 RAM

#### Английский

In [17]:
sourceid="en"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
with open(source, encoding='utf-8') as f:
    text = f.readlines()
text = (" ".join([i.strip() for i in text]))
text = split_text_into_sentences(text, language=sourceid)

result = []
n = 0
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
for sent in text:
    start = time.time() # начало замера времени
    n += 1
    sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
    print(n, sent[:50], end='...')
    data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
    answer = requests.post(url, headers=headers, data=data).json()['message']
    test = requests.post(url, headers=headers, data=data).json()['test']
    measure["preproc"].append(test["preproc"])
    measure["token"].append(test["token"])
    measure["generate"].append(test["generate"])
    measure["decode"].append(test["decode"])
    measure["memory"].append(test["memory"])

    end = time.time() # окончание замера времени
    all_time += (end - start) # фиксация затраченного времени
    print(answer[:50])
    result.append(answer)
    
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 This case was resolved with the same evidence that...Это дело было урегулировано с использованием тех ж
2 If justice is a form of dissuasion to prevent peop...Если правосудие является одной из форм устрашения 
3 Authorities must publicly denounce violence on jou...Власти должны публично осудить насилие в отношении
4 Indeed, [the government of] Colombia adopted a pro...Действительно, [правительство] Колумбии приняло пр
5 But the program has a mixed record, especially aft...Но эта программа имеет смешанный послужной список,
6 In September, the People's Party presented a draft...В сентябре Народная партия представила на рассмотр
7 While the law is still under review, its proponent...Хотя этот закон все еще находится на рассмотрении,
8 After a deadly shooting in Almaty, the country's l...После смертельной стрельбы в Алма-Ате, крупнейшем 
9 I also learned a lot along the way about the craft...Я также многому научился на пути к искусству литер
10 I'm still learning, I believe....Я все еще 

#### Арабский

In [18]:
sourceid="ar"
targetid="ru"
source = f'input.{sourceid}'
target = f'api-output.en-{targetid}'
url = 'http://10.239.16.89:60001/api/translation' 
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}

result = []
all_time = 0
measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
with open(source, encoding='utf-8') as f:
    n = 0
    for i in f:
        start = time.time() # начало замера времени
        sent = i.strip()
        n += 1
        sent = sent.encode('utf-8').decode('latin-1', 'ignore')
        print(n, sent[:50], end='...')
        data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        print(answer[:50])
        result.append(answer)
        
with open(f'new_{target}', 'w', encoding='utf-8') as f:
    f.write("\n".join(result) + '\n')

preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

print('*' * 106)
print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
print(f'Затрачено времени: {all_time:.2f} сек.')

1 Ø­ÙÙØª ØªÙÙ Ø§ÙÙØ¶ÙØ© Ø¨Ø°Ø§Øª Ø§ÙØ¯ÙÙÙ...Это дело было раскрыто теми же доказательствами, ч
2 ÙØ¬Ø¨ Ø¹ÙÙ Ø§ÙØ¯ÙÙØ© Ø£Ù ØªÙØ¯ÙÙ Ø¹ÙØ§Ù...Государство должно публично осудить насилие в отно
3 ØªØ¨ÙØª [Ø§ÙØ­ÙÙÙØ©] Ø§ÙÙÙÙÙÙØ¨ÙØ© Ø¨Ø...Колумбийское [правительство] приняло программу защ
4 ÙÙ Ø³Ø¨ØªÙØ¨Ø±/Ø£ÙÙÙÙØ ÙØ¯ÙÙ Ø­Ø²Ø¨ Ø§Ù...В сентябре Народная партия представила парламенту 
5 ÙÙØ¹ Ø£Ù Ø§ÙÙØ§ÙÙÙ ÙØ§ ÙØ²Ø§Ù ÙÙØ¯ Ø§...Хотя этот закон все еще находится на рассмотрении,
6 ÙØ¹ÙØ¨ Ø­Ø¯ÙØ« Ø·ÙÙ ÙØ§Ø±ÙÙ ÙÙÙØª ÙÙ ...После того, как в Алматы, крупнейшем городе страны
7 ØªØ¹ÙÙØª Ø£ÙØ¶ÙØ§ Ø§ÙÙØ«ÙØ± ÙÙ Ø§ÙØ£Ø´Ù...Я также многому научился на пути к литературному п
8 ÙØ£Ø¤ÙÙ Ø£ÙÙÙ ÙØ§ Ø£Ø²Ø§Ù Ø£ØªØ¹ÙÙ...И я верю, что все еще учусь.
9 ÙØ§Ù Ø§ÙØ¬Ø§ÙØ¨ Ø§ÙØªØ¬Ø§Ø±Ù Ø·Ø±ÙÙÙØ§ Ø´...Коммерческая сторона была очень труднодоступной с 
10 Ø¨Ø¯Ø£Øª ÙØ´Ø±ÙØ¹ Ø§ÙÙØªØ§Ø¨ Ø¹Ø§Ù 2017 ÙÙÙ...Проект книги был н

## Эксперименты с числом CPU

### Оценка влияния CPU

In [25]:
def en_ru(title='************'):
    sourceid="en"
    targetid="ru"
    source = f'input.{sourceid}'
    target = f'api-output.en-{targetid}'
    url = 'http://10.239.16.89:60001/api/translation' 
    headers = {'accept': 'application/json',
               'Content-Type': 'application/json',
    }
    with open(source, encoding='utf-8') as f:
        text = f.readlines()
    text = (" ".join([i.strip() for i in text]))
    text = split_text_into_sentences(text, language=sourceid)

    result = []
    n = 0
    all_time = 0
    measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
    for sent in text:
        start = time.time() # начало замера времени
        n += 1
        sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
        data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
        answer = requests.post(url, headers=headers, data=data).json()['message']
        test = requests.post(url, headers=headers, data=data).json()['test']
        measure["preproc"].append(test["preproc"])
        measure["token"].append(test["token"])
        measure["generate"].append(test["generate"])
        measure["decode"].append(test["decode"])
        measure["memory"].append(test["memory"])

        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        result.append(answer)

    preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
    token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
    generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
    decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
    memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

    print('EN-RU\n', title)
    print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
    print(f'Затрачено времени: {all_time:.2f} сек.')

In [26]:
def ar_ru(title='************'):
    sourceid="ar"
    targetid="ru"
    source = f'input.{sourceid}'
    target = f'api-output.en-{targetid}'
    url = 'http://10.239.16.89:60001/api/translation' 
    headers = {'accept': 'application/json',
               'Content-Type': 'application/json',
    }

    result = []
    all_time = 0
    measure = {"preproc": [], "token": [], "generate": [], "decode": [], "memory_start": [], "memory_end": [], "memory": []}
    with open(source, encoding='utf-8') as f:
        n = 0
        for i in f:
            start = time.time() # начало замера времени
            sent = i.strip()
            n += 1
            sent = sent.encode('utf-8').decode('latin-1', 'ignore')
            data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
            answer = requests.post(url, headers=headers, data=data).json()['message']
            test = requests.post(url, headers=headers, data=data).json()['test']
            measure["preproc"].append(test["preproc"])
            measure["token"].append(test["token"])
            measure["generate"].append(test["generate"])
            measure["decode"].append(test["decode"])
            measure["memory"].append(test["memory"])

            end = time.time() # окончание замера времени
            all_time += (end - start) # фиксация затраченного времени
            result.append(answer)

    preproc = f'{sum(measure["preproc"] * 1000):.2f} ms, среднее: {sum(measure["preproc"]) * 1000 / len(measure["preproc"]):.3f} ms'
    token = f'{sum(measure["token"] * 1000):.2f} ms, среднее: {sum(measure["token"]) * 1000 / len(measure["token"]):.3f} ms'
    generate = f'{sum(measure["generate"] * 1000):.2f} ms, среднее: {sum(measure["generate"]) * 1000 / len(measure["generate"]):.3f} ms'
    decode = f'{sum(measure["decode"] * 1000):.2f} ms, среднее: {sum(measure["decode"]) * 1000 / len(measure["decode"]):.3f} ms'
    memory = f'{sum(measure["memory"]) / (1024 * 1024):.2f} Mb, среднее: {sum(measure["memory"]) / len(measure["memory"]) / (1024 * 1024):.5f} Mb'

    print('AR-RU\n', title)
    print(f'preproc: {preproc}\ntoken: {token}\ngenerate: {generate}\ndecode: {decode}\nmemory: {memory}\nпредложений: {len(measure["preproc"])}')
    print(f'Затрачено времени: {all_time:.2f} сек.')

In [30]:
title = '1 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 1 CPU
preproc: 6.54 ms, среднее: 0.096 ms
token: 128.83 ms, среднее: 1.895 ms
generate: 2499352.79 ms, среднее: 36755.188 ms
decode: 4021.01 ms, среднее: 59.133 ms
memory: 982.14 Mb, среднее: 14.44330 Mb
предложений: 68
Затрачено времени: 5037.24 сек.
AR-RU
 1 CPU
preproc: 6.73 ms, среднее: 0.118 ms
token: 213.02 ms, среднее: 3.737 ms
generate: 2351672.69 ms, среднее: 41257.416 ms
decode: 3246.89 ms, среднее: 56.963 ms
memory: 1118.96 Mb, среднее: 19.63096 Mb
предложений: 57
Затрачено времени: 4708.34 сек.


In [31]:
title = '2 CPU'
en_ru(title)
print('-' * 25)
ar_ru(title)

EN-RU
 2 CPU
preproc: 5.89 ms, среднее: 0.087 ms
token: 111.31 ms, среднее: 1.637 ms
generate: 1079303.48 ms, среднее: 15872.110 ms
decode: 1239.77 ms, среднее: 18.232 ms
memory: 1075.05 Mb, среднее: 15.80957 Mb
предложений: 68
Затрачено времени: 2158.52 сек.
-------------------------
AR-RU
 2 CPU
preproc: 5.19 ms, среднее: 0.091 ms
token: 115.00 ms, среднее: 2.018 ms
generate: 1024312.51 ms, среднее: 17970.395 ms
decode: 1121.12 ms, среднее: 19.669 ms
memory: 928.47 Mb, среднее: 16.28893 Mb
предложений: 57
Затрачено времени: 2049.86 сек.


In [32]:
title = '3 CPU'
en_ru(title)
print('-' * 25)
ar_ru(title)

EN-RU
 3 CPU
preproc: 4.79 ms, среднее: 0.070 ms
token: 100.63 ms, среднее: 1.480 ms
generate: 609500.08 ms, среднее: 8963.236 ms
decode: 641.38 ms, среднее: 9.432 ms
memory: 1015.39 Mb, среднее: 14.93216 Mb
предложений: 68
Затрачено времени: 1218.79 сек.
-------------------------
AR-RU
 3 CPU
preproc: 4.04 ms, среднее: 0.071 ms
token: 290.89 ms, среднее: 5.103 ms
generate: 573290.83 ms, среднее: 10057.734 ms
decode: 385.32 ms, среднее: 6.760 ms
memory: 1058.79 Mb, среднее: 18.57518 Mb
предложений: 57
Затрачено времени: 1148.05 сек.


In [33]:
title = '4 CPU'
en_ru(title)
print('-' * 25)
ar_ru(title)

EN-RU
 4 CPU
preproc: 3.83 ms, среднее: 0.056 ms
token: 71.40 ms, среднее: 1.050 ms
generate: 332945.86 ms, среднее: 4896.263 ms
decode: 1121.19 ms, среднее: 16.488 ms
memory: 926.18 Mb, среднее: 13.62023 Mb
предложений: 68
Затрачено времени: 671.96 сек.
-------------------------
AR-RU
 4 CPU
preproc: 3.35 ms, среднее: 0.059 ms
token: 70.21 ms, среднее: 1.232 ms
generate: 323899.22 ms, среднее: 5682.442 ms
decode: 788.74 ms, среднее: 13.838 ms
memory: 976.35 Mb, среднее: 17.12891 Mb
предложений: 57
Затрачено времени: 646.32 сек.


In [34]:
title = '5 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 5 CPU
preproc: 3.46 ms, среднее: 0.051 ms
token: 66.46 ms, среднее: 0.977 ms
generate: 270910.53 ms, среднее: 3983.978 ms
decode: 496.11 ms, среднее: 7.296 ms
memory: 1046.01 Mb, среднее: 15.38247 Mb
предложений: 68
Затрачено времени: 542.30 сек.
AR-RU
 5 CPU
preproc: 3.21 ms, среднее: 0.056 ms
token: 66.39 ms, среднее: 1.165 ms
generate: 258176.15 ms, среднее: 4529.406 ms
decode: 454.63 ms, среднее: 7.976 ms
memory: 1172.41 Mb, среднее: 20.56853 Mb
предложений: 57
Затрачено времени: 519.85 сек.


In [35]:
title = '10 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 10 CPU
preproc: 2.73 ms, среднее: 0.040 ms
token: 50.39 ms, среднее: 0.741 ms
generate: 108374.31 ms, среднее: 1593.740 ms
decode: 278.38 ms, среднее: 4.094 ms
memory: 1074.21 Mb, среднее: 15.79716 Mb
предложений: 68
Затрачено времени: 220.04 сек.
AR-RU
 10 CPU
preproc: 2.87 ms, среднее: 0.050 ms
token: 52.20 ms, среднее: 0.916 ms
generate: 106243.00 ms, среднее: 1863.912 ms
decode: 174.66 ms, среднее: 3.064 ms
memory: 1097.59 Mb, среднее: 19.25596 Mb
предложений: 57
Затрачено времени: 212.99 сек.


In [36]:
title = '15 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 15 CPU
preproc: 2.76 ms, среднее: 0.041 ms
token: 48.96 ms, среднее: 0.720 ms
generate: 65604.37 ms, среднее: 964.770 ms
decode: 130.92 ms, среднее: 1.925 ms
memory: 1084.70 Mb, среднее: 15.95146 Mb
предложений: 68
Затрачено времени: 133.22 сек.
AR-RU
 15 CPU
preproc: 2.49 ms, среднее: 0.044 ms
token: 46.46 ms, среднее: 0.815 ms
generate: 62316.16 ms, среднее: 1093.266 ms
decode: 148.56 ms, среднее: 2.606 ms
memory: 867.43 Mb, среднее: 15.21813 Mb
предложений: 57
Затрачено времени: 125.73 сек.


In [37]:
title = '20 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 20 CPU
preproc: 2.38 ms, среднее: 0.035 ms
token: 46.25 ms, среднее: 0.680 ms
generate: 45732.32 ms, среднее: 672.534 ms
decode: 113.69 ms, среднее: 1.672 ms
memory: 1098.44 Mb, среднее: 16.15349 Mb
предложений: 68
Затрачено времени: 93.52 сек.
AR-RU
 20 CPU
preproc: 2.48 ms, среднее: 0.043 ms
token: 48.36 ms, среднее: 0.849 ms
generate: 43217.90 ms, среднее: 758.209 ms
decode: 103.55 ms, среднее: 1.817 ms
memory: 1146.10 Mb, среднее: 20.10698 Mb
предложений: 57
Затрачено времени: 88.47 сек.


In [38]:
title = '25 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 25 CPU
preproc: 2.39 ms, среднее: 0.035 ms
token: 52.76 ms, среднее: 0.776 ms
generate: 45620.25 ms, среднее: 670.886 ms
decode: 119.57 ms, среднее: 1.758 ms
memory: 1147.31 Mb, среднее: 16.87224 Mb
предложений: 68
Затрачено времени: 93.29 сек.
AR-RU
 25 CPU
preproc: 2.30 ms, среднее: 0.040 ms
token: 46.58 ms, среднее: 0.817 ms
generate: 43122.09 ms, среднее: 756.528 ms
decode: 111.37 ms, среднее: 1.954 ms
memory: 1061.54 Mb, среднее: 18.62356 Mb
предложений: 57
Затрачено времени: 87.90 сек.


In [41]:
title = '30 CPU'
en_ru(title)
ar_ru(title)

EN-RU
 30 CPU
preproc: 2.38 ms, среднее: 0.035 ms
token: 46.10 ms, среднее: 0.678 ms
generate: 45468.98 ms, среднее: 668.662 ms
decode: 117.23 ms, среднее: 1.724 ms
memory: 1240.76 Mb, среднее: 18.24644 Mb
предложений: 68
Затрачено времени: 92.64 сек.
AR-RU
 30 CPU
preproc: 2.52 ms, среднее: 0.044 ms
token: 47.88 ms, среднее: 0.840 ms
generate: 43858.25 ms, среднее: 769.443 ms
decode: 111.75 ms, среднее: 1.961 ms
memory: 1043.38 Mb, среднее: 18.30489 Mb
предложений: 57
Затрачено времени: 88.77 сек.
